In [1]:
import pandas as pd 
import json 
import datetime
import numpy as np
import matplotlib.pyplot as plt 
from sklearn import preprocessing
from geoencoder.grid_encoder import GridEncoder 
from geoencoder.location import Location


In [2]:
location_data = pd.read_csv("geocoded.tsv", sep="\t")

In [3]:
blockchain_info = json.load(open("blockchaininfo.json"))

In [4]:
del location_data["coin"]

In [5]:
location_data.set_index("address", inplace=True)

In [6]:
location_data.head()

,bottom,left,top,right
address,,,,
1000000000000000000000000000,52.169746,12.463552,52.862511,14.305487
1000000000000000000000000000000,52.169746,12.463552,52.862511,14.305487
104fb073f9a131f2cab49184bb864ca2,43.306789,-80.157295,43.999554,-78.608101
106d4648661d49e16d103b071e26617e,30.554756,120.872208,31.869980,122.244728
10lvumPW24UNLoQU1RSseyIDwGgjQ,35.365032,-32.044041,71.734970,67.004044


In [7]:
location_data_dic = location_data.to_dict("index")

In [8]:
addresses =[] 
hours = []

In [9]:
i=0
#print(len(blockchain_info))
for val in blockchain_info.values():
    if val != None:
        address = val["address"]
        txs = val["txs"]
        if len(txs) != 0:
            time = txs[0]["time"]
            parsed_time = datetime.datetime.utcfromtimestamp(time)
            hour = parsed_time.hour
            minute = parsed_time.minute

            addresses.append(address)
            hours.append(hour)

In [10]:
le = preprocessing.LabelEncoder()

In [11]:
hour_vectors = le.fit_transform(hours)

In [12]:
num_points = len(hours)

In [13]:
training = list(zip(hours, addresses))[:int(0.6*num_points)]
validation = list(zip(hour_vectors, addresses))[int(0.6*num_points):int(0.8*num_points)]
testing = list(zip(hour_vectors, addresses))[int(0.8*num_points):]

In [14]:
def get_hour_location_prob(data, resolution):
    grid_encoder = GridEncoder(resolution)
    hour_to_mapping = {}
    hour_counts = {}
    for hour, address in data:
        box= location_data_dic[address]
        location =Location(box["bottom"], box["top"], box["left"], box["right"])
        encoding = grid_encoder.encode(location)
        #print(np.sum(np.sum(encoding)))
        if hour not in hour_to_mapping:
            hour_counts[hour] =1 
            hour_to_mapping[hour] = encoding
        else:
            hour_counts[hour]+=1
            hour_to_mapping[hour] += encoding
            #print(np.sum(hour_to_mapping[hour])
    for hour in hour_to_mapping:
        hour_to_mapping[hour] /= hour_counts[hour]
    return hour_to_mapping


In [15]:
def get_hour_raw(data):
    hour_to_lat = {}
    hour_to_long = {}
    for hour, address in data:
        hour_to_lat.setdefault(hour, default=[])
        hourrr_to_long.setdefault(hour, default=[])
        box= location_data_dic[address]
        lat = (box["bottom"]+box["top"])/2
        long = (box["left"]+box["right"])/2
        hour_to_lat[hour].append(lat)
        hour_to_long[hour].append(log)

In [64]:
training_hour_location_prob = get_hour_location_prob(training, 18)
gen_dist = np.array([np.array(val) for val in training_hour_location_prob.values()])
gen_dist = np.sum(gen_dist, axis=0)/24
print(gen_dist.shape)
validation_hour_location_prob = get_hour_location_prob(validation, 18)


(10, 20)


In [54]:
print(np.sum(np.sum(validation_hour_location_prob[1])))

1.0


In [65]:
gen_discrepancy = 0
hour_discrepancy =0 
for hour in validation_hour_location_prob:
    hour_discrepancy += np.sum(np.sum(np.abs(training_hour_location_prob[hour]- validation_hour_location_prob[hour])))
    diff = np.sum(np.abs(gen_dist- validation_hour_location_prob[hour]))
    gen_discrepancy += diff


In [66]:
print(gen_discrepancy)
print(hour_discrepancy)

25.4019230276
25.2716379961


In [ ]:
plt.pcolormesh(training_hour_location_prob[8])
plt.show()